In [18]:
import requests
import pandas as pd

# Your Ticketmaster API key
apikey = "B1887CGuksd5FGGlfb3cybG1ALtsYOQO"

# Base URL for the API
base_url = "https://app.ticketmaster.com/discovery/v2/events.json"

# Parameters
params = {
    "apikey": apikey,
    "countryCode": "ES",
    "size": 200,  
    "page": 0,   
}

# Container for all data
all_events = []
current_page = 0
page_max = 5

In [19]:
# Fetch data
while page_max > current_page:
    print(f"Fetching page {params['page']}...")

    # Make the request
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()

        # Extract events from the response
        events = data.get("_embedded", {}).get("events", [])
        if not events:  # Break loop if no more events
            print("No more events found.")
            break

        # Extract relevant details
        for event in events:
            event_info = {
                "event_id": event.get("id"),
                "event_name": event.get("name"),
                "event_type": event.get("type"),
                "event_url": event.get("url"),
                "locale": event.get("locale"),
                "start_date": event.get("dates", {}).get("start", {}).get("localDate"),
                "start_time": event.get("dates", {}).get("start", {}).get("localTime"),
                "timezone": event.get("dates", {}).get("timezone"),
                "status": event.get("dates", {}).get("status", {}).get("code"),
            }

            # Extract venue information
            venues = event.get("_embedded", {}).get("venues", [])
            for venue in venues:
                venue_info = {
                    "venue_id": venue.get("id"),
                    "venue_name": venue.get("name"),
                    "venue_city": venue.get("city", {}).get("name"),
                    "venue_state": venue.get("state", {}).get("name"),
                    "venue_country": venue.get("country", {}).get("name"),
                    "venue_address": venue.get("address", {}).get("line1"),
                    "venue_postal_code": venue.get("postalCode"),
                    "venue_latitude": venue.get("location", {}).get("latitude"),
                    "venue_longitude": venue.get("location", {}).get("longitude"),
                }

            # Extract classification (genre) information
            classifications = event.get("classifications", [])
            for classification in classifications:
                classification_info = {
                    "segment_name": classification.get("segment", {}).get("name"),
                    "genre_name": classification.get("genre", {}).get("name"),
                    "sub_genre_name": classification.get("subGenre", {}).get("name"),
                }

            # Combine all extracted information
            combined_info = {**event_info, **venue_info, **classification_info}
            all_events.append(combined_info)

        # Check if there are more pages
        current_page += 1

        # Move to the next page
        params["page"] += 1
    else:
        print(f"Error: {response.status_code}, {response.text}")
        break


Fetching page 0...
Fetching page 1...
No more events found.


In [20]:
all_events

[{'event_id': 'Z698xZ2qZa17W',
  'event_name': 'Paul McCartney: GOT BACK TOUR',
  'event_type': 'event',
  'event_url': 'https://www.ticketmaster.es/event/paul-mccartney-got-back-tour-tickets/41277?language=en-us',
  'locale': 'en-us',
  'start_date': '2024-12-09',
  'start_time': '21:00:00',
  'timezone': 'Europe/Madrid',
  'status': 'onsale',
  'venue_id': 'Z198xZ2qZkek',
  'venue_name': 'WiZink Center',
  'venue_city': 'Madrid',
  'venue_state': 'Madrid',
  'venue_country': 'Spain',
  'venue_address': 'Av. Felipe II, s/n',
  'venue_postal_code': '28009',
  'venue_latitude': '40.42394',
  'venue_longitude': '-3.6758',
  'segment_name': 'Music',
  'genre_name': 'Rock',
  'sub_genre_name': 'Pop'},
 {'event_id': 'Z698xZ2qZa1Ad',
  'event_name': 'Paul McCartney: GOT BACK TOUR',
  'event_type': 'event',
  'event_url': 'https://www.ticketmaster.es/event/paul-mccartney-got-back-tour-tickets/41283?language=en-us',
  'locale': 'en-us',
  'start_date': '2024-12-10',
  'start_time': '21:00:00',

In [21]:
# Save the data to a CSV file
if all_events:
    df = pd.DataFrame(all_events)
    df.to_csv("ticketmaster_cleaned_events.csv", index=False)
    print("Data saved to 'ticketmaster_cleaned_events.csv'.")
else:
    print("No data collected.")

Data saved to 'ticketmaster_cleaned_events.csv'.
